In [1]:
import os

def abc_arr_creater():
    letter = 'a'
    arr = list()
    count = 1
    while (count < 27):
        if letter != 'w':
            arr.append(letter)
        letter = chr(ord(letter) + 1)
        count += 1
    return arr

def create_dataset_folders():
    dataset_arr = ['train', 'test', 'validation']
    alphabet_arr = abc_arr_creater()
    diractory = 'lips_cut_dataset'
    parent_dir = ''
    path = os.path.join(parent_dir, diractory)
    for set_dir in dataset_arr:
        set_path = os.path.join(path, set_dir)
        for letter in alphabet_arr:
            letter_dir_path = os.path.join(set_path, letter)
            os.makedirs(letter_dir_path)
            print('directory ' + letter + ' in ' + set_dir + ' created')
        

In [2]:
create_dataset_folders()

directory a in train created
directory b in train created
directory c in train created
directory d in train created
directory e in train created
directory f in train created
directory g in train created
directory h in train created
directory i in train created
directory j in train created
directory k in train created
directory l in train created
directory m in train created
directory n in train created
directory o in train created
directory p in train created
directory q in train created
directory r in train created
directory s in train created
directory t in train created
directory u in train created
directory v in train created
directory x in train created
directory y in train created
directory z in train created
directory a in test created
directory b in test created
directory c in test created
directory d in test created
directory e in test created
directory f in test created
directory g in test created
directory h in test created
directory i in test created
directory j in test cre

In [3]:
import json
import cv2

def get_video_files():  # get the names of the video files to a list
    video_files = list()
    path_to_video = 'front'  # url to dir where the dataset video locate
    for file in os.listdir(path_to_video):
        if file.endswith('mov'):  # if file end with mov add it to the list
            video_files.append(file)
    return video_files

In [4]:
def get_json_files():  # get the jason files to a list
    jason_files = list()
    path_to_jason_files = 'alignment'  # url path for the text of the dataset
    for pos_json in os.listdir(path_to_jason_files):  # loop at the dir and look for jason flies
        if pos_json.endswith('.json'):
            jason_files.append(pos_json)
    return jason_files


In [5]:
def get_valid_videos(jsonFiles, temp_videoNames):
    valid_videos = []
    for file in temp_videoNames:
        if file in jsonFiles:
            valid_videos.append(file + '.mov')
    return valid_videos

In [6]:
def get_video_text(jason_list, videofile):
    video_name = videofile.replace('.mov', '.json')  # name of the jason file
    jason_data = ''
    url = 'alignment'  # location of the file
    for file in jason_list:
        if file == video_name:  # if file name as the name of the video then open it as jason file
            with open('alignment/' + video_name) as json_file:
                jason_data = json.load(json_file)  # jason_data is now a jason file
                break
    return jason_data

In [7]:
def findLetterTime(data, video_name): # find the offset and the end offset of the letter
    counter = 0
    start_time = ''
    end_time = ''
    for p in data[video_name]:
        if '_B' in str(p['phone']):
            counter += 1
        if counter == 4:
            start_time = p['offset']
            counter += 1
        if counter > 4 and '_E' in str(p['phone']):
            end_time = p['offset'] + p['duration']
            break
        if '_S' in str(p['phone']) and counter >= 3:
            start_time = p['offset']
            end_time = p['offset'] + p['duration']
            break
    return start_time,end_time

In [8]:
def runAndSaveFrames(cap, path, videoName, dircount):
    count = 0
    frame_counter = 0
    while (cap.isOpened()):
        ret, frame = cap.read()  # read the frame
        if ret == True and frame_counter <= 14:
            cv2.imwrite(path + '/frame%d.jpg' % count, frame)  # save the current frame as framenumber.jpg
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
        count += 1
        frame_counter += 1

In [12]:
def procces(json_files, validVideofiles): # run the videos and divide them to test, train, validation 
    dataset_arr = ['train','test', 'validation']
    dircount = 0
    for setdir in dataset_arr:
        if setdir == 'train':
            limit = 4000 # in the train there will be frames from the first 4000 videos 
        elif setdir == 'test':
            limit = 5000
        else:
            limit = 5319
        while dircount < limit:
            file = validVideofiles[dircount]
            data = get_video_text(json_files, file)  # get the video text
            videoName = file.split('.')[0]
            letterStart, letterEnd = findLetterTime(data, videoName)  # get the letter start time
            letter = str(dircount) + videoName.split('_')[2][3]  # letter and counter
            letter_class = videoName.split('_')[2][3]  # the letter class in the set
            cap = cv2.VideoCapture('front/' + file)  # get the video file
            constant = 222.2222223  # number of melsec/100 to add to get the requested 3 frames
            os.mkdir('updated_dataset/' + setdir + '/' + letter_class + '/' + letter) # make the dir number_letter in the set
            
            # the loop was here
            offset_mel = letterStart * 1000
            duration_mel = (letterEnd - letterStart) * 1000 + constant
            cap.set(cv2.CAP_PROP_POS_MSEC, offset_mel) # run the video from the letter start point
            path = 'updated_dataset/' + setdir + '/' + letter_class + '/' + letter # the path to save the frames
            runAndSaveFrames(cap, path, videoName, dircount)
        #  Release everything if job is finished
            cap.release()
            cv2.destroyAllWindows()
            print('directory ' + letter + ' saved in ' + path )
            dircount += 1

In [11]:

def main():
    tempVideo_files = get_video_files()
    json_files = get_json_files()  # get all the texts
    jsonFilesNames = []
    temp_videoNames = []
    for jsonfile in json_files:
        jsonNamesSplit = jsonfile.split('.')
        jsonFilesNames.append(jsonNamesSplit[0])
    for video in tempVideo_files:
        videoSplit = video.split('.')
        temp_videoNames.append(videoSplit[0])

    validVideofiles = get_valid_videos(jsonFilesNames, temp_videoNames)
    procces(json_files, validVideofiles)
    

In [12]:
main()

s52_p_srwe5s.mov
